In [23]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

In [24]:
#Adj R square
def adj_r2_score(model,X_test, y_test,):
    y_pred = model.predict(X_test)
    # model.coefs_ doesn't exist
    adj = 1 - float(len(y_test)-1)/(len(y_test)-model.n_features_-1) * \
            (1 - r2_score(y_test,y_pred))
    return adj

# Evalute random search
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    MSE = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
#    r2_adjusted = adj_r2_score(model, X_test, y_test)
    print('Model Validation')
    print('MSE: {0}'.format(MSE))
    print('R^2: {0}'.format(r2))
#    print('R^2 Adjusted: {0}'.format(r2_adjusted))

In [25]:
# read csv file
dataset = pd.read_csv('./csvfiles/output_training.csv')
dataset['merchanttype'] = dataset['merchanttype'].astype('category')
dataset['merchantname'] = dataset['merchantname'].astype('category')
dataset['topads'] = dataset['topads'].replace((1,0), ('yes', 'no'))
dataset['topads'] = dataset['topads'].astype('category')
# drop column 'id' and 'prodname'
dataset = dataset.drop(['id', 'prodname', 'merchantname', 'actualrevcount'], axis=1)
# some 'revpictotal' has 9999 value and most of them have
# 'actualrevcount' = 10, only 5a986371b8a9f712ce73da7f has 119 'actualrevcount'
# replace 9999 with np.nan
dataset['revpictotal'] = dataset['revpictotal'].replace(9999, np.nan)
# data imputation
# mean technique
meanway = dataset['revpictotal'].mean()
dataset['revpictotal'] = dataset['revpictotal'].fillna(meanway)

In [26]:
dataset.head()

,merchanttype,topads,cashback,cashbackval,price,prodrating,reviewcount,negreview,posreview,answerscnt,otheragreemean,ratingmosthelpful,possentiment,negsentiment,sentipolarity,reviewersrep,revpictotal,prodpicstotal,ranking
0,biasa,no,0.0,0.0,28877,4.8,3200,0.005330,0.994670,1301,0,5,3,-1,1,1,545.0,23,41580.0
1,gold,no,0.0,0.0,103000,4.6,3100,0.006742,0.993258,2336,0,5,3,-1,1,1,653.0,48,41233.5
2,gold,no,0.0,0.0,2500,4.8,2200,0.004630,0.995370,280,0,5,3,-1,1,1,494.0,10,41571.0
3,gold,no,0.0,0.0,570000,4.8,2100,0.001577,0.998423,1417,0,5,3,-1,1,1,341.0,26,41052.5
4,gold,no,0.0,0.0,159000,4.5,2100,0.020270,0.979730,2541,0,4,3,-1,1,1,403.0,28,40821.5


In [27]:
dataset.describe()

,cashback,cashbackval,price,prodrating,reviewcount,negreview,posreview,answerscnt,otheragreemean,ratingmosthelpful,possentiment,negsentiment,sentipolarity,reviewersrep,revpictotal,prodpicstotal,ranking
count,41596.000000,41596.000000,4.159600e+04,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.000000,41596.0,41596.000000,41596.000000,41596.000000
mean,0.001125,228.700575,2.489824e+05,4.725113,44.338903,0.010768,0.989232,35.129363,0.002909,1.515843,3.252957,-1.064069,0.998365,1.0,10.123301,0.592557,20798.500000
std,0.006985,6191.385698,9.518654e+05,0.219873,85.406325,0.037130,0.037130,76.364433,0.056472,2.193405,0.585073,0.245956,0.045441,0.0,15.468910,1.477846,12007.630252
min,0.000000,0.000000,1.000000e+02,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,1.000000,-3.000000,-1.000000,1.0,0.000000,0.000000,86.000000
25%,0.000000,0.000000,1.600000e+04,4.600000,11.000000,0.000000,1.000000,7.000000,0.000000,0.000000,3.000000,-1.000000,1.000000,1.0,5.000000,0.000000,10221.500000
50%,0.000000,0.000000,6.000000e+04,4.800000,22.000000,0.000000,1.000000,16.000000,0.000000,0.000000,3.000000,-1.000000,1.000000,1.0,8.000000,0.000000,20768.000000
75%,0.000000,0.000000,1.699000e+05,4.900000,45.000000,0.000000,1.000000,35.000000,0.000000,4.000000,4.000000,-1.000000,1.000000,1.0,10.123301,1.000000,31214.000000
max,0.050000,687500.000000,5.500000e+07,5.000000,3200.000000,0.666667,1.000000,2541.000000,4.000000,5.000000,5.000000,-1.000000,1.000000,1.0,653.000000,48.000000,41596.000000


In [28]:
dftrain = dataset.values

In [29]:
dftrain

array([['biasa', 'no', 0.0, ..., 545.0, 23, 41580.0],
       ['gold', 'no', 0.0, ..., 653.0, 48, 41233.5],
       ['gold', 'no', 0.0, ..., 494.0, 10, 41571.0],
       ...,
       ['gold', 'no', 0.0, ..., 3.0, 1, 3058.5],
       ['biasa', 'no', 0.0, ..., 3.0, 0, 23707.0],
       ['biasa', 'no', 0.0, ..., 2.0, 0, 1051.5]], dtype=object)

In [30]:
dftrain.shape

(41596, 19)

In [31]:
# Encoding categorical data ('merchanttype' and 'topads')
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
# 'merchanttype'
dftrain[:, 0] = labelencoder1.fit_transform(dftrain[:, 0])
# 'topads'
dftrain[:, 1] = labelencoder2.fit_transform(dftrain[:, 1])
# onehotencoder for both 'merchanttype'
onehotencoder = OneHotEncoder(categorical_features=[0])
dftrain = onehotencoder.fit_transform(dftrain).toarray()

In [32]:
dftrain.shape

(41596, 21)

In [35]:
dftrain[:,0:4]

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [11]:
# dummy variables for merchanttype (3-1)
# avoiding dummy variable trap
dftrain = dftrain[:, 1:]

In [12]:
dftrain.shape

(41596, 20)

In [13]:
sc = StandardScaler()

In [14]:
dftrain = sc.fit_transform(dftrain)

In [15]:
X = dftrain[:, :-1].copy()

In [16]:
y = dftrain[:, 19].copy()

In [17]:
X.shape

(41596, 19)

In [18]:
X[0]
# merchanttype2, merchanttype3, topads, cashback, 
# cashbackval, price, prodrating, reviewcount, 
# negreview, posreview, answercnt, otheragreemean, 
# ratingmosthelpful, possentiment, negsentiment, sentipolarity, 
# reviewersrep, revpictotal, prodpicstotal

array([-1.63709649e+00, -4.52509671e-02, -2.48649798e-01, -1.61001634e-01,
       -3.69389560e-02, -2.31238606e-01,  3.40596826e-01,  3.69492426e+01,
       -1.46458720e-01,  1.46458720e-01,  1.65769037e+01, -5.15120936e-02,
        1.58848886e+00, -4.32356634e-01,  2.60491723e-01,  3.59761571e-02,
        0.00000000e+00,  3.45779465e+01,  1.51624155e+01])

In [19]:
# splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# 1 - Decision Tree

In [13]:
from sklearn.tree import DecisionTreeRegressor

In [28]:
regressor = DecisionTreeRegressor(min_samples_split=5, random_state=42, max_depth=5)

In [37]:
parameters = [{'min_samples_split': range(2, 15),
              'max_depth':[5, 6, 7, 8, 9],
              'min_samples_leaf': [4, 5, 6],
              'max_features': ['auto', 'log2', 'sqrt']}
             ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=10,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [38]:
# model fitting using grid search parameter
grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_
result = grid_search.cv_results_

Fitting 10 folds for each of 585 candidates, totalling 5850 fits
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.3s


[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=5 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=4, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=7 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=4, total=   0.1s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=6, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_sample

[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=9, total=   0.1s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=11 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=11 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=11, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=12 
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=11 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_sp

[CV] max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=3 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=14, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=3 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=2, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=2 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=2, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=3 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=2, total=   0.1s
[CV] max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=2 
[CV] max_depth=5, max_features=auto, min_samples_leaf=5, min_samples_split=2 
[CV]  max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=14, total=   0.1s
[CV]  max_depth=5, max_features=auto, min_samples_leaf=5, min_sam

...

[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=11 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=9, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=11 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_s

[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=3 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=14, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=5, min_samples_split=14, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=2, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_l

[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=6, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=7, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_le

[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11 
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=12 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=10, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV] max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=12 
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=11, total=   0.0s
[CV]  max_depth=9, max_features=sqrt, min_samples_leaf=6, min_samples_split=12, total=   0.0s
[CV] max_depth=9, max_feat

[Parallel(n_jobs=-1)]: Done 5850 out of 5850 | elapsed:   52.4s finished


In [39]:
best_parameters

{'max_depth': 8,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 12}

In [42]:
best_estimator.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=12, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [43]:
# cross validation
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)

accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV] ................................................. , total=   0.1s
[CV] ................................................. , total=   0.1s
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished


(-0.2617081409302261, 0.0055112584466852094)

In [44]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 0.2521550580659747
R^2: 0.7470690155798495
R^2 Adjusted: 0.7464900169408155


In [46]:
joblib.dump(best_estimator, './training/regressor_destree_new.pkl')

['./training/regressor_destree_new.pkl']

# 2 - Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [53]:
# robust regression using RANSAC
from sklearn.linear_model import RANSACRegressor
ransac = RANSACRegressor(LinearRegression(),
        max_trials=300,
        min_samples=50,
        loss='absolute_loss',
        residual_threshold=5.0,
        random_state=42)
ransac.fit(X_train, y_train)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

In [54]:
# model evaluation
from sklearn.metrics import mean_squared_error, r2_score
y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)
y_train_pred_ransac = ransac.estimator_.predict(X_train)
y_test_pred_ransac = ransac.estimator_.predict(X_test)

In [56]:
def evaluate(y_train_pred, y_train, y_test_pred, y_test):
    print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
    print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [57]:
evaluate(y_train_pred, y_train, y_test_pred, y_test)

MSE train: 0.731, test: 0.739
R^2 train: 0.269, test: 0.258


In [58]:
evaluate(y_train_pred_ransac, y_train, y_test_pred_ransac, y_test)

MSE train: 0.800, test: 0.829
R^2 train: 0.200, test: 0.168


# 3 - SVR

In [59]:
from sklearn.svm import SVR
regressor = SVR(kernel='poly')

In [60]:
# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=10,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [61]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total= 3.3min
[CV] C=5, kern

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 17.1min


[CV] ............................... C=5, kernel=linear, total= 9.2min
[CV] C=10, kernel=linear .............................................
[CV] ............................... C=5, kernel=linear, total= 9.6min
[CV] C=100, kernel=linear ............................................
[CV] ............................... C=5, kernel=linear, total=10.0min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total=16.2min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total=16.7min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total=15.2min
[CV] C=100, kernel=linear ............................................
[CV] .............................. C=10, kernel=linear, total=16.1min
[CV] C=100, kernel=linear ............................................
[CV] .

[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.4min
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.4min
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.5min
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.4min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total= 1.5min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] ....................... C=5, gamma=0.5, kernel=rbf, total= 4.3min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] ............................ C=100, kernel=linear, total=119.3min
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] .

[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.7min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.7min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.6min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.7min
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] ............................ C=100, kernel=linear, total=134.5min
[CV] C=10, gamma=0.5, kernel=rbf .....................................


[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 172.6min


[CV] .................... C=5, gamma=0.0001, kernel=rbf, total= 1.8min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ............................ C=100, kernel=linear, total=140.6min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 7.9min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.1, kernel=rbf, total= 3.5min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 7.9min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 8.2min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total= 7.9min
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] .

[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=34.7min
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=35.6min
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=36.0min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=35.8min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=35.5min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=37.6min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=17.7min
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .

[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 259.8min finished


In [63]:
best_parameters

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [65]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total= 2.5min
[CV] ................................................. , total= 2.6min
[CV] ................................................. , total= 2.7min
[CV] ................................................. , total= 2.9min
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.9min remaining:  2.9min


[CV] ................................................. , total= 3.0min
[CV] ................................................. , total= 3.0min
[CV] ................................................. , total= 3.0min
[CV] ................................................. , total= 3.0min


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.0min finished


(-0.3740563933153908, 0.007888615923619315)

In [71]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 0.3704861604446768
R^2: 0.6283737871687032


In [73]:
joblib.dump(best_estimator, './training/regressor_svr_new.pkl')

['./training/regressor_svr_new.pkl']

# 4 - Multi-Layer Perceptron

In [3]:
from sklearn.neural_network import MLPRegressor
regressor = MLPRegressor(random_state=42)

In [26]:
# read csv file
dataset = pd.read_csv('./csvfiles/output_training.csv')
dataset['merchanttype'] = dataset['merchanttype'].astype('category')
dataset['merchantname'] = dataset['merchantname'].astype('category')
dataset['topads'] = dataset['topads'].replace((1,0), ('yes', 'no'))
dataset['topads'] = dataset['topads'].astype('category')
# drop column 'id' and 'prodname'
dataset = dataset.drop(['id', 'prodname', 'merchantname', 'actualrevcount'], axis=1)
# some 'revpictotal' has 9999 value and most of them have
# 'actualrevcount' = 10, only 5a986371b8a9f712ce73da7f has 119 'actualrevcount'
# replace 9999 with np.nan
dataset['revpictotal'] = dataset['revpictotal'].replace(9999, np.nan)
# data imputation
# mean technique
meanway = dataset['revpictotal'].mean()
dataset['revpictotal'] = dataset['revpictotal'].fillna(meanway)

dftrain = dataset.values

# Encoding categorical data ('merchanttype' and 'topads')
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
# 'merchanttype'
dftrain[:, 0] = labelencoder1.fit_transform(dftrain[:, 0])
# 'topads'
dftrain[:, 1] = labelencoder2.fit_transform(dftrain[:, 1])
# onehotencoder for both 'merchanttype'
onehotencoder = OneHotEncoder(categorical_features=[0])
dftrain = onehotencoder.fit_transform(dftrain).toarray()

# dummy variables for merchanttype (3-1)
# avoiding dummy variable trap
dftrain = dftrain[:, 1:]

sc = MinMaxScaler()
dftrain = sc.fit_transform(dftrain)
X = dftrain[:, :-1].copy()
y = dftrain[:, 19].copy()

# splitting the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [7]:
# GRID SEARCH
parameters = [{
    'hidden_layer_sizes': [(24, ), (32,), (64,), (128, 128)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam', 'lbfgs'],
    'batch_size': ['auto', 8, 16, 32, 64, 128],
    'early_stopping': [True]
    } 
    ]

grid = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=10,
        n_jobs=-1,
        refit='neg_mean_squared_error',
        verbose=2
    )

In [8]:
grid_result = grid.fit(X_train, y_train)

best_parameters = grid_result.best_params_
best_estimator = grid_result.best_estimator_
result = grid_result.cv_results_

Fitting 10 folds for each of 96 candidates, totalling 960 fits
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam 
[CV] activ

/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam, total= 1.6min
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 


/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=adam, total= 1.6min
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=lbfgs, total= 1.1min
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=lbfgs, total= 1.1min
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(24,), solver=lbfgs, total= 1.1min
[CV] activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=logistic, batch_size=auto, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  5

...

[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  41.4s
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  36.4s
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  35.3s
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=32, early_stopping=True, hidden_layer_sizes=(128, 128), solver=lbfgs, total= 6.6min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  24.1s
[CV] activation=relu, batch_size=64, early

[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.5min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.6min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.5min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam, total= 1.8min
[CV] activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam, total= 1.1min
[CV] activation=

[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=relu, batch_size=64, early_stopping=True, hidden_layer_sizes=(128, 128), solver=lbfgs, total= 7.3min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  53.1s
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  25.7s
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=adam, total=  26.3s
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(32,), solver=lbfgs 
[CV]  activation=relu, batch_size=64, early_stoppin

[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.6min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.5min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.5min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.6min
[CV] activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(128, 128), solver=adam 
[CV]  activation=relu, batch_size=128, early_stopping=True, hidden_layer_sizes=(64,), solver=lbfgs, total= 1.4min
[CV] activation=re

[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 204.3min finished


In [9]:
best_parameters

{'activation': 'relu',
 'batch_size': 32,
 'early_stopping': True,
 'hidden_layer_sizes': (128, 128),
 'solver': 'adam'}

In [10]:
evaluate(best_estimator, X_train, y_train)

Model Validation
MSE: 0.029417895910017144
R^2: 0.6486957547962959


In [11]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 0.0292605387009415
R^2: 0.6492336185603937


In [12]:
# cross validation
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)

accuracies.mean(), accuracies.std() 

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=  56.6s
[CV] ................................................. , total= 2.1min
[CV] ................................................. , total= 2.2min
[CV] ................................................. , total= 2.5min
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.5min remaining:  2.5min


[CV] ................................................. , total= 2.9min
[CV] ................................................. , total= 3.0min
[CV] ................................................. , total= 3.0min
[CV] ................................................. , total= 3.0min
[CV] ................................................. , total= 3.0min


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.0min finished


(-0.02864765226971443, 0.003619732538963186)

In [13]:
accuracies

array([-0.02665666, -0.0389718 , -0.02742315, -0.02736324, -0.02683053,
       -0.03001339, -0.02844764, -0.02689234, -0.02820292, -0.02567487])

In [14]:
# cross validation
accuracies = cross_val_score(estimator=best_estimator, X=X_test, y=y_test, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)

accuracies.mean(), accuracies.std() 

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=  23.8s
[CV] ................................................. , total=  26.3s
[CV] ................................................. , total=  28.4s
[CV] ................................................. , total=  31.0s
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   32.1s remaining:   32.1s


[CV] ................................................. , total=  33.1s
[CV] ................................................. , total=  33.5s
[CV] ................................................. , total=  36.8s
[CV] ................................................. , total=  38.6s
[CV] ................................................. , total=  38.8s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   38.9s finished


(-0.035779020265082846, 0.0019257325581409658)

In [15]:
accuracies

array([-0.03697241, -0.03413341, -0.03512504, -0.03320973, -0.03333689,
       -0.03526957, -0.036084  , -0.03592643, -0.03937704, -0.0383557 ])

In [16]:
# save fitted model to file
joblib.dump(best_estimator, './training/regressor_mlp_new.pkl')

['./training/regressor_mlp_new.pkl']

In [24]:
np.random.seed(42)

In [20]:
estimator = joblib.load('./training/regressor_destree_new.pkl')

In [21]:
evaluate(estimator, X_train, y_train)

Model Validation
MSE: 0.23782085627294125
R^2: 0.7623595126405909


In [22]:
evaluate(estimator, X_test, y_test)

Model Validation
MSE: 0.2521550580659747
R^2: 0.7470690155798495
